
# Drought Features: Weekly Aggregation for FIPS 6107

This notebook builds a **feature dataset for drought modeling** by aggregating weather variables over rolling windows, using only the **weekly-scored dates** as targets.

## What it does
- Loads a CSV of daily (or sub-daily) weather data with a `score` column available on **weekly** rows.
- Filters to **FIPS 6107**.
- For every weekly (scored) date, computes rolling-window aggregates:
  - **Precipitation** sums (7/30/90/180 days)
  - **Temperature** means, plus **T2M_MAX** maxima
  - **Humidity proxies** (QV2M, T2MDEW, T2MWET mean)
  - **Wind** (10m, 50m) means/max/min-mean/range-mean
  - **Pressure** means
  - Additional derived metrics (precip deficits, temp anomalies, VPD proxies)
- Exports a new dataset with one row per weekly (scored) date.

> Tip: If you want to change the FIPS or add/remove features, look for the **Parameters** and **Feature Engineering** sections below.



## Parameters
Update these to point to your input data and desired output file name.


In [ ]:

import pandas as pd
import numpy as np

# --- File paths ---
INPUT_CSV = './data/archive/train_timeseries/train_timeseries.csv'        # <--- update to your file path
OUTPUT_CSV = './data/generated_data/fips_6107_drought_features.csv'

# --- Target FIPS ---
TARGET_FIPS = 6107  # Kern County, CA (example)



## Load and prepare the data
- Reads the input CSV
- Parses the `date` column to datetime
- Filters to the target **FIPS**
- Sorts by date to ensure correct rolling-window logic


In [ ]:

# Load your data
df = pd.read_csv(INPUT_CSV)

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

# Filter for TARGET_FIPS only
df_filtered = df[df['fips'] == TARGET_FIPS].copy()

# Sort by date to ensure proper rolling calculations
df_filtered = df_filtered.sort_values('date').reset_index(drop=True)

# Identify rows with scores (weekly observations)
scored_dates = df_filtered[df_filtered['score'].notna()]['date'].values

print(f"Total rows in input: {len(df):,}")
print(f"Rows for FIPS {TARGET_FIPS}: {len(df_filtered):,}")
print(f"Weekly (scored) dates found: {len(scored_dates):,}")
df_filtered.head()



## Feature engineering on weekly (scored) dates
For each **target (weekly)** date, compute aggregates over the trailing windows:
- **7, 30, 90, 180 days** (some families only use a subset).
We build a new table with one row per weekly date.


In [ ]:

# Create the new dataset - start with scored weeks only
result_rows = []

for target_date in scored_dates:
    target_date = pd.Timestamp(target_date)
    
    # Get the score for this week
    score_value = df_filtered.loc[df_filtered['date'] == target_date, 'score'].values[0]
    
    # Create masks for different rolling windows ending on target_date (exclusive of the lower bound)
    mask_7d = (df_filtered['date'] <= target_date) & (df_filtered['date'] > target_date - pd.Timedelta(days=7))
    mask_30d = (df_filtered['date'] <= target_date) & (df_filtered['date'] > target_date - pd.Timedelta(days=30))
    mask_90d = (df_filtered['date'] <= target_date) & (df_filtered['date'] > target_date - pd.Timedelta(days=90))
    mask_180d = (df_filtered['date'] <= target_date) & (df_filtered['date'] > target_date - pd.Timedelta(days=180))
    
    # Initialize row with basic info
    row = {'fips': TARGET_FIPS, 'date': target_date, 'score': score_value}
    
    # PRECIPITATION FEATURES - Sum over windows
    row['prec_sum_7d'] = df_filtered.loc[mask_7d, 'PRECTOT'].sum()
    row['prec_sum_30d'] = df_filtered.loc[mask_30d, 'PRECTOT'].sum()
    row['prec_sum_90d'] = df_filtered.loc[mask_90d, 'PRECTOT'].sum()
    row['prec_sum_180d'] = df_filtered.loc[mask_180d, 'PRECTOT'].sum()
    
    # TEMPERATURE FEATURES - Mean over windows
    for period, mask in [('7d', mask_7d), ('30d', mask_30d), ('90d', mask_90d), ('180d', mask_180d)]:
        row[f't2m_mean_{period}'] = df_filtered.loc[mask, 'T2M'].mean()
        row[f't2m_max_mean_{period}'] = df_filtered.loc[mask, 'T2M_MAX'].mean()
        row[f't2m_min_mean_{period}'] = df_filtered.loc[mask, 'T2M_MIN'].mean()
        row[f't2m_range_mean_{period}'] = df_filtered.loc[mask, 'T2M_RANGE'].mean()
        row[f'ts_mean_{period}'] = df_filtered.loc[mask, 'TS'].mean()
    
    # TEMPERATURE MAX for heatwave detection (7d and 30d only)
    row['t2m_max_7d'] = df_filtered.loc[mask_7d, 'T2M_MAX'].max()
    row['t2m_max_30d'] = df_filtered.loc[mask_30d, 'T2M_MAX'].max()
    
    # HUMIDITY PROXIES - Mean over 7/30/90d
    for period, mask in [('7d', mask_7d), ('30d', mask_30d), ('90d', mask_90d)]:
        row[f'qv2m_mean_{period}'] = df_filtered.loc[mask, 'QV2M'].mean()
        row[f't2mdew_mean_{period}'] = df_filtered.loc[mask, 'T2MDEW'].mean()
        row[f't2mwet_mean_{period}'] = df_filtered.loc[mask, 'T2MWET'].mean()
    
    # WIND FEATURES - Mean and Max over 7/30d (evaporative demand)
    for period, mask in [('7d', mask_7d), ('30d', mask_30d)]:
        # 10m wind
        row[f'ws10m_mean_{period}'] = df_filtered.loc[mask, 'WS10M'].mean()
        row[f'ws10m_max_{period}'] = df_filtered.loc[mask, 'WS10M_MAX'].max()
        row[f'ws10m_min_mean_{period}'] = df_filtered.loc[mask, 'WS10M_MIN'].mean()
        row[f'ws10m_range_mean_{period}'] = df_filtered.loc[mask, 'WS10M_RANGE'].mean()
        
        # 50m wind
        row[f'ws50m_mean_{period}'] = df_filtered.loc[mask, 'WS50M'].mean()
        row[f'ws50m_max_{period}'] = df_filtered.loc[mask, 'WS50M_MAX'].max()
        row[f'ws50m_min_mean_{period}'] = df_filtered.loc[mask, 'WS50M_MIN'].mean()
        row[f'ws50m_range_mean_{period}'] = df_filtered.loc[mask, 'WS50M_RANGE'].mean()
    
    # PRESSURE FEATURES - Mean over 7/30d
    row['ps_mean_7d'] = df_filtered.loc[mask_7d, 'PS'].mean()
    row['ps_mean_30d'] = df_filtered.loc[mask_30d, 'PS'].mean()
    
    # ADDITIONAL DROUGHT-RELEVANT FEATURES
    # Precipitation deficit (compare recent to longer-term average)
    row['prec_deficit_30v90d'] = row['prec_sum_30d'] - (row['prec_sum_90d'] / 3)
    row['prec_deficit_7v30d'] = row['prec_sum_7d'] - (row['prec_sum_30d'] / 4.3)
    
    # Temperature anomaly (recent vs longer-term)
    row['temp_anomaly_7v90d'] = row['t2m_mean_7d'] - row['t2m_mean_90d']
    row['temp_anomaly_30v180d'] = row['t2m_mean_30d'] - row['t2m_mean_180d']
    
    # Vapor pressure deficit proxy (temperature - dewpoint)
    row['vpd_proxy_7d'] = row['t2m_mean_7d'] - row['t2mdew_mean_7d']
    row['vpd_proxy_30d'] = row['t2m_mean_30d'] - row['t2mdew_mean_30d']
    
    result_rows.append(row)

# Create final dataframe
df_drought_features = pd.DataFrame(result_rows)
print(f"Dataset created with {len(df_drought_features)} weekly observations")
print(f"Total features: {len(df_drought_features.columns)}")
df_drought_features.head()



## Export results
Saves the engineered features to CSV for downstream modeling.


In [ ]:

# Save to CSV
df_drought_features.to_csv(OUTPUT_CSV, index=False)

print(f"Saved features to: {OUTPUT_CSV}")
print(f"Columns ({len(df_drought_features.columns)}):")
print(df_drought_features.columns.tolist())



## Quick QA checks
Missing values, dtypes, and a preview.


In [ ]:

print("/nDataset info:")
print(df_drought_features.info())

print("/nMissing values (total):", df_drought_features.isnull().sum().sum())

# Peek
df_drought_features.head(10)



## Notes & Next Steps
- If the dataset is very large, consider vectorizing with rolling windows or using a time-index and `pd.Series.rolling` for speed.
- You can generalize this notebook to loop over **multiple FIPS** and then concatenate results.
- For modeling, consider normalizing/standardizing features and adding **lag** versions of key metrics.
